
Loading the CIFAR-10 Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision import models

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Augmentation and Normalization for CIFAR-10
# Enhanced Data Augmentation and Normalization for CIFAR-10
transform = {
    'train': transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}


# Load CIFAR-10 Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform['train'])
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform['test'])


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class IntermediateBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_layers, kernel_size=3, activation=nn.ReLU()):
        super(IntermediateBlock, self).__init__()
        self.conv_layers = nn.ModuleList([])
        for _ in range(num_layers):
            self.conv_layers.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size//2),
                nn.BatchNorm2d(out_channels),
                activation
            ))
        self.fc = nn.Linear(in_channels, num_layers)

    def forward(self, x):
        m = x.mean([2, 3])  # Compute mean separately for each color channel
        a = torch.softmax(self.fc(m), dim=1)  # Calculate 'a' using a fully connected layer
        x_prime = sum(a[:, i].view(-1, 1, 1, 1) * conv(x) for i, conv in enumerate(self.conv_layers))

        return x_prime



In [ ]:
class OutputBlock(nn.Module):
    def __init__(self, in_channels, num_classes, hidden_layers=[]):
        super(OutputBlock, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc_layers = nn.ModuleList()

        # Add fully connected layers
        input_size = in_channels
        for hidden_size in hidden_layers:
            self.fc_layers.append(nn.Linear(input_size, hidden_size))
            input_size = hidden_size
        self.fc_layers.append(nn.Linear(input_size, num_classes))

    def forward(self, x):
        # Reduce spatial dimensions to 1x1 using adaptive average pooling
        x = self.avgpool(x)

        # Flatten the tensor to a vector
        x = torch.flatten(x, 1)

        # Pass through fully connected layers
        for fc_layer in self.fc_layers:
            x = fc_layer(x)

        return x

In [ ]:
# Hyperparameters
epochs = 30
learning_rate = 0.001
batch_size = 128
weight_decay = 1e-4

In [ ]:
class Model_1(nn.Module):
    def __init__(self, num_classes=10):
        super(Model_1, self).__init__()
        self.layer1 = IntermediateBlock(3, 64, 2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer2 = IntermediateBlock(64, 128, 2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer3 = IntermediateBlock(128, 256, 3)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer4 = IntermediateBlock(256, 512, 3)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.output_block = OutputBlock(512, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.pool1(x)

        x = self.layer2(x)
        x = self.pool2(x)

        x = self.layer3(x)
        x = self.pool3(x)

        x = self.layer4(x)
        x = self.pool4(x)

        x = self.output_block(x)
        return x

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model_1().to(device)
input_tensor = torch.randn(1, 3, 32, 32).to(device)  # Example input tensor
output = model(input_tensor)
print("Output shape:", output.shape)


Output shape: torch.Size([1, 10])


/opt/conda/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1702400440653/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Define training and testing functions

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.squeeze())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets.squeeze()).sum().item()

    return train_loss / len(train_loader), 100. * correct / total

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets.squeeze()).sum().item()

    return test_loss / len(test_loader), 100. * correct / total

In [ ]:
# Training and testing loop
num_epochs = 50
train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)
    scheduler.step()

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")


Epoch [1/50], Train Loss: 1.3713, Train Acc: 50.39%, Test Loss: 1.1872, Test Acc: 58.57%
Epoch [2/50], Train Loss: 1.0059, Train Acc: 64.27%, Test Loss: 1.0465, Test Acc: 62.88%
Epoch [3/50], Train Loss: 0.8668, Train Acc: 69.61%, Test Loss: 1.3041, Test Acc: 56.74%
Epoch [4/50], Train Loss: 0.7872, Train Acc: 72.53%, Test Loss: 0.7186, Test Acc: 75.18%
Epoch [5/50], Train Loss: 0.7223, Train Acc: 75.11%, Test Loss: 0.9181, Test Acc: 69.50%
Epoch [6/50], Train Loss: 0.6881, Train Acc: 76.07%, Test Loss: 0.9531, Test Acc: 69.53%
Epoch [7/50], Train Loss: 0.6481, Train Acc: 77.35%, Test Loss: 0.6856, Test Acc: 76.81%
Epoch [8/50], Train Loss: 0.6220, Train Acc: 78.41%, Test Loss: 0.6477, Test Acc: 78.03%
Epoch [9/50], Train Loss: 0.5910, Train Acc: 79.56%, Test Loss: 0.6732, Test Acc: 77.94%
Epoch [10/50], Train Loss: 0.5712, Train Acc: 80.00%, Test Loss: 0.7050, Test Acc: 77.01%
Epoch [11/50], Train Loss: 0.5453, Train Acc: 81.02%, Test Loss: 0.6103, Test Acc: 79.85%
Epoch [12/50], Trai

In [ ]:
import matplotlib.pyplot as plt
import torch

# Plotting
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), epoch_losses, label='Average Epoch Loss')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, epochs + 1), test_accuracies, label='Testing Accuracy')
plt.title('Training and Testing Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

print(f'Final Testing Accuracy: {best_accuracy:.2f}%')

